In [2]:
from datasets import load_dataset
import pandas as pd
import numpy as np

c:\Users\fredd\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Grocery_and_Gourmet_Food", trust_remote_code=True)
print(dataset['full'][0])

{'rating': 5.0, 'title': 'Excellent!  Yummy!', 'text': 'Excellent!! Yummy!  Great with other foods and great alone.', 'images': [], 'asin': 'B00CM36GAQ', 'parent_asin': 'B00CM36GAQ', 'user_id': 'AFKZENTNBQ7A7V7UXW5JJI6UGRYQ', 'timestamp': 1587854482395, 'helpful_vote': 0, 'verified_purchase': True}


In [4]:
df_reviews = dataset['full'].to_pandas()

In [5]:
df_reviews.head(5)

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Excellent! Yummy!,Excellent!! Yummy! Great with other foods and...,[],B00CM36GAQ,B00CM36GAQ,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1587854482395,0,True
1,5.0,Delicious!!! Yum!,Excellent! The best! I use it with my beef a...,[],B074J5WVYH,B0759B7KLH,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1587854400380,0,True
2,5.0,"Extremely Delicious, but expensive imo",These are very tasty. They are extremely soft ...,[],B079TRNVHX,B079TRNVHX,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1587853224527,1,True
3,5.0,Delicious!,My favorite!,[],B07194LN2Z,B07194LN2Z,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1581313319614,0,True
4,5.0,Great taste,Great for making brownies and crinkle cookies.,[],B005CD4196,B005CD4196,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1581313294965,7,True


In [6]:
df_reviews.shape

(14318520, 10)

In [7]:
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Grocery_and_Gourmet_Food", trust_remote_code=True)
print(dataset['full'][0])

{'main_category': 'Grocery', 'title': 'Dark Roast Pure Coffee', 'average_rating': 4.7, 'rating_number': 9, 'features': [], 'description': [], 'price': 'None', 'images': {'hi_res': ['https://m.media-amazon.com/images/I/81ucKSAeinL._SL1500_.jpg'], 'large': ['https://m.media-amazon.com/images/I/51LY15EJoFL.jpg'], 'thumb': ['https://m.media-amazon.com/images/I/51LY15EJoFL._SX38_SY50_CR,0,0,38,50_.jpg'], 'variant': ['MAIN']}, 'videos': {'title': ['YouCut_20220Cafe Du Monde Coffee Chicory, 15 Ounce Ground coffee', 'CDM New Orleans'], 'url': ['https://www.amazon.com/vdp/08a1120516ab4b2184f486f940c0de45?ref=dp_vse_rvc_0', 'https://www.amazon.com/vdp/0ab93bf7ed5646f68d1fa8f00cd30160?ref=dp_vse_rvc_1'], 'user_id': ['/shop/influencer-d62c132e', '']}, 'store': 'Luzianne', 'categories': ['Grocery & Gourmet Food', 'Beverages', 'Coffee'], 'details': '{"Brand": "Luzianne", "Item Form": "Ground", "Caffeine Content": "caffeinated", "Roast Level": "dark_roast", "Unit Count": "13.00 Ounce", "Package Dimen

In [8]:
df_meta = dataset['full'].to_pandas()

In [9]:
df_meta.head(5)

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,Grocery,Dark Roast Pure Coffee,4.7,9,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,{'title': ['YouCut_20220Cafe Du Monde Coffee C...,Luzianne,"[Grocery & Gourmet Food, Beverages, Coffee]","{""Brand"": ""Luzianne"", ""Item Form"": ""Ground"", ""...",B00NE08WM6,None,None,None
1,Grocery,PICARAS Galletas Peruanas Bañadas en Chocolate...,4.5,12,[2 Bags of PICARAS Galletas Bañadas en Chocola...,[2 Bags of PICARAS Galletas Bañadas en Chocola...,15.99,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Winters,"[Grocery & Gourmet Food, Breads & Bakery, Cook...","{""Flavor"": ""chocolate"", ""Unit Count"": ""12.00 C...",B084Q13Q5Q,None,None,None
2,Grocery,Chipped Beef and Gravy By Patterson's - Great ...,3.2,5,[Six (6) 8-ounce cans of delicious chipped bee...,[Delicious corned beef gravy is ready when you...,None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Pattersons,"[Grocery & Gourmet Food, Pantry Staples, Canne...","{""Package Dimensions"": ""8.4 x 5.8 x 3.4 inches...",B00KBRUYVM,None,None,None
3,Grocery,Asher's Sugar Free Milk Chocolate Cordial Cher...,5.0,6,[],[],29.99,"{'hi_res': [None], 'large': ['https://m.media-...",{'title': ['No Sugar Added Chocolate Truffles'...,Generic,"[Grocery & Gourmet Food, Snacks & Sweets, Choc...","{""Brand"": ""Generic"", ""Item Form"": ""Ball"", ""Cho...",B0BN4PW255,None,None,None
4,Grocery,Messmer Peppermint 25 bags (6er pack),3.5,5,[],[],29.99,"{'hi_res': [None, None], 'large': ['https://m....","{'title': [], 'url': [], 'user_id': []}",Messmer,[],"{""Brand"": ""Messmer"", ""Item Form"": ""Teabags"", ""...",B06X9DC27H,None,None,None


In [10]:
df_meta.shape

(603274, 16)

In [11]:
df_reviews = df_reviews.drop(columns=['images'], axis=1)
df_reviews

,rating,title,text,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Excellent! Yummy!,Excellent!! Yummy! Great with other foods and...,B00CM36GAQ,B00CM36GAQ,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1587854482395,0,True
1,5.0,Delicious!!! Yum!,Excellent! The best! I use it with my beef a...,B074J5WVYH,B0759B7KLH,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1587854400380,0,True
2,5.0,"Extremely Delicious, but expensive imo",These are very tasty. They are extremely soft ...,B079TRNVHX,B079TRNVHX,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1587853224527,1,True
3,5.0,Delicious!,My favorite!,B07194LN2Z,B07194LN2Z,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1581313319614,0,True
4,5.0,Great taste,Great for making brownies and crinkle cookies.,B005CD4196,B005CD4196,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1581313294965,7,True
...,...,...,...,...,...,...,...,...,...
14318515,5.0,Excellent option!,I'm not someone who usually seeks out vegan or...,B0BL3CFF91,B0BL3CFF91,AHZCFE4SGY7KUH45GUM3NUEBIVMA,1684217579701,2,True
14318516,5.0,Perfect for Baby Shower Cake,This arrived exactly as pictured and was a hug...,B06ZYFZJ8Z,B06ZYFZJ8Z,AHZCFE4SGY7KUH45GUM3NUEBIVMA,1520959816272,0,True
14318517,5.0,Great product,Delivered quickly. True to description. Very ...,B07DK3WNRJ,B07DK3WNRJ,AEAFQYSOX43C77Z36HQH5I6IRV2Q,1560724206928,0,True
14318518,5.0,I Love it,So beautiful and I love it. I recommend it.,B0714KTNVM,B0714KTNVM,AF4N6QB3QXLDRGTSWRV35T7AAH2Q,1507325205106,0,True


In [12]:
df_meta = df_meta.drop(columns=['images'], axis=1)
df_meta

,main_category,title,average_rating,rating_number,features,description,price,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,Grocery,Dark Roast Pure Coffee,4.7,9,[],[],None,{'title': ['YouCut_20220Cafe Du Monde Coffee C...,Luzianne,"[Grocery & Gourmet Food, Beverages, Coffee]","{""Brand"": ""Luzianne"", ""Item Form"": ""Ground"", ""...",B00NE08WM6,None,None,None
1,Grocery,PICARAS Galletas Peruanas Bañadas en Chocolate...,4.5,12,[2 Bags of PICARAS Galletas Bañadas en Chocola...,[2 Bags of PICARAS Galletas Bañadas en Chocola...,15.99,"{'title': [], 'url': [], 'user_id': []}",Winters,"[Grocery & Gourmet Food, Breads & Bakery, Cook...","{""Flavor"": ""chocolate"", ""Unit Count"": ""12.00 C...",B084Q13Q5Q,None,None,None
2,Grocery,Chipped Beef and Gravy By Patterson's - Great ...,3.2,5,[Six (6) 8-ounce cans of delicious chipped bee...,[Delicious corned beef gravy is ready when you...,None,"{'title': [], 'url': [], 'user_id': []}",Pattersons,"[Grocery & Gourmet Food, Pantry Staples, Canne...","{""Package Dimensions"": ""8.4 x 5.8 x 3.4 inches...",B00KBRUYVM,None,None,None
3,Grocery,Asher's Sugar Free Milk Chocolate Cordial Cher...,5.0,6,[],[],29.99,{'title': ['No Sugar Added Chocolate Truffles'...,Generic,"[Grocery & Gourmet Food, Snacks & Sweets, Choc...","{""Brand"": ""Generic"", ""Item Form"": ""Ball"", ""Cho...",B0BN4PW255,None,None,None
4,Grocery,Messmer Peppermint 25 bags (6er pack),3.5,5,[],[],29.99,"{'title': [], 'url': [], 'user_id': []}",Messmer,[],"{""Brand"": ""Messmer"", ""Item Form"": ""Teabags"", ""...",B06X9DC27H,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603269,Grocery,LVEUD Happy Birthday Cake Topper Black Font Go...,5.0,7,[],[],None,"{'title': [], 'url': [], 'user_id': []}",None,"[Grocery & Gourmet Food, Pantry Staples, Cooki...","{""Is Discontinued By Manufacturer"": ""No"", ""Pro...",B07NJV1FMQ,None,None,None
603270,Grocery,Sugar Free Hard Candy Factor Zero Espresso Cof...,5.0,2,[],[],None,"{'title': [], 'url': [], 'user_id': []}",Z FACTOR ZERO,[],"{""Package Dimensions"": ""8.9 x 7.83 x 2.91 inch...",B07RS4F1J4,None,None,None
603271,Grocery,"Tre Mulini: ""Mezze Penne Rigate"" Durum Wheat S...",5.0,1,[Pasta made of first quality 100% durum semoli...,"[Pasta is a staple of Mediterranean diet, that...",None,"{'title': [], 'url': [], 'user_id': []}",Tre Mulini,"[Grocery & Gourmet Food, Pantry Staples, Pasta...","{""Brand"": ""Tre Mulini"", ""Variety"": ""Penne Riga...",B00KLF7I1S,None,None,None
603272,Grocery,KaBloom PRIME NEXT DAY DELIVERY - 12 Fresh Pur...,3.2,3,[],[This cheerful bouquet of fresh cut purple ros...,39.24,"{'title': [], 'url': [], 'user_id': []}",KaBloom,[],"{""Brand"": ""KaBloom"", ""Color"": ""Purple"", ""Numbe...",B071F3LMGC,None,None,None


In [13]:
df_meta = df_meta.drop(columns=['videos'], axis=1)
df_meta

,main_category,title,average_rating,rating_number,features,description,price,store,categories,details,parent_asin,bought_together,subtitle,author
0,Grocery,Dark Roast Pure Coffee,4.7,9,[],[],None,Luzianne,"[Grocery & Gourmet Food, Beverages, Coffee]","{""Brand"": ""Luzianne"", ""Item Form"": ""Ground"", ""...",B00NE08WM6,None,None,None
1,Grocery,PICARAS Galletas Peruanas Bañadas en Chocolate...,4.5,12,[2 Bags of PICARAS Galletas Bañadas en Chocola...,[2 Bags of PICARAS Galletas Bañadas en Chocola...,15.99,Winters,"[Grocery & Gourmet Food, Breads & Bakery, Cook...","{""Flavor"": ""chocolate"", ""Unit Count"": ""12.00 C...",B084Q13Q5Q,None,None,None
2,Grocery,Chipped Beef and Gravy By Patterson's - Great ...,3.2,5,[Six (6) 8-ounce cans of delicious chipped bee...,[Delicious corned beef gravy is ready when you...,None,Pattersons,"[Grocery & Gourmet Food, Pantry Staples, Canne...","{""Package Dimensions"": ""8.4 x 5.8 x 3.4 inches...",B00KBRUYVM,None,None,None
3,Grocery,Asher's Sugar Free Milk Chocolate Cordial Cher...,5.0,6,[],[],29.99,Generic,"[Grocery & Gourmet Food, Snacks & Sweets, Choc...","{""Brand"": ""Generic"", ""Item Form"": ""Ball"", ""Cho...",B0BN4PW255,None,None,None
4,Grocery,Messmer Peppermint 25 bags (6er pack),3.5,5,[],[],29.99,Messmer,[],"{""Brand"": ""Messmer"", ""Item Form"": ""Teabags"", ""...",B06X9DC27H,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603269,Grocery,LVEUD Happy Birthday Cake Topper Black Font Go...,5.0,7,[],[],None,None,"[Grocery & Gourmet Food, Pantry Staples, Cooki...","{""Is Discontinued By Manufacturer"": ""No"", ""Pro...",B07NJV1FMQ,None,None,None
603270,Grocery,Sugar Free Hard Candy Factor Zero Espresso Cof...,5.0,2,[],[],None,Z FACTOR ZERO,[],"{""Package Dimensions"": ""8.9 x 7.83 x 2.91 inch...",B07RS4F1J4,None,None,None
603271,Grocery,"Tre Mulini: ""Mezze Penne Rigate"" Durum Wheat S...",5.0,1,[Pasta made of first quality 100% durum semoli...,"[Pasta is a staple of Mediterranean diet, that...",None,Tre Mulini,"[Grocery & Gourmet Food, Pantry Staples, Pasta...","{""Brand"": ""Tre Mulini"", ""Variety"": ""Penne Riga...",B00KLF7I1S,None,None,None
603272,Grocery,KaBloom PRIME NEXT DAY DELIVERY - 12 Fresh Pur...,3.2,3,[],[This cheerful bouquet of fresh cut purple ros...,39.24,KaBloom,[],"{""Brand"": ""KaBloom"", ""Color"": ""Purple"", ""Numbe...",B071F3LMGC,None,None,None


In [14]:
df_meta['categories'] = df_meta['categories'].apply(lambda x: tuple(x) if isinstance(x, (list, np.ndarray)) else x)
df_meta[['categories', 'parent_asin']].drop_duplicates()

,categories,parent_asin
0,"(Grocery & Gourmet Food, Beverages, Coffee)",B00NE08WM6
1,"(Grocery & Gourmet Food, Breads & Bakery, Cook...",B084Q13Q5Q
2,"(Grocery & Gourmet Food, Pantry Staples, Canne...",B00KBRUYVM
3,"(Grocery & Gourmet Food, Snacks & Sweets, Choc...",B0BN4PW255
4,(),B06X9DC27H
...,...,...
603269,"(Grocery & Gourmet Food, Pantry Staples, Cooki...",B07NJV1FMQ
603270,(),B07RS4F1J4
603271,"(Grocery & Gourmet Food, Pantry Staples, Pasta...",B00KLF7I1S
603272,(),B071F3LMGC


In [15]:
df_carb = df_meta[df_meta['title'].str.contains('low carb', case=False, na=False)]
parent_asin_list_carb = df_carb['parent_asin'].drop_duplicates().to_list()
len(parent_asin_list_carb)

4403

In [16]:
df_carb.head(5)

,main_category,title,average_rating,rating_number,features,description,price,store,categories,details,parent_asin,bought_together,subtitle,author
25,Grocery,"Yacona Organic Stevia Alternative, Keto Food Y...",4.1,277,[HEALTHY SWEETENER: Having trouble finding a s...,[],None,Yacona,"(Grocery & Gourmet Food, Pantry Staples, Cooki...","{""Package Dimensions"": ""3.23 x 2.83 x 2.5 inch...",B082VQCS5W,None,None,None
34,Grocery,Quevos Egg White Chips - Original Low Carb Egg...,3.9,11,[THE FIRST EGG WHITE CRISPS - We knew that pot...,[],None,Quevos,"(Grocery & Gourmet Food, Snacks & Sweets, Snac...","{""Package Dimensions"": ""8.39 x 7.99 x 5.2 inch...",B09KSCP4MS,None,None,None
83,Grocery,Quevos Egg White Chips - The Original Low Carb...,3.5,115,[THE FIRST EGG WHITE CRISPS - We knew that pot...,[Quevos are the world first chips made from eg...,None,Quevos,"(Grocery & Gourmet Food, Snacks & Sweets, Snac...","{""Package Dimensions"": ""9.45 x 7.01 x 6.14 inc...",B089S8G1VG,None,None,None
153,Grocery,Certified Keto & Gluten Free All Purpose Bread...,3.5,8,"[Bake delicious, fresh out of the oven gluten-...",[],15.99,Fox Hill Kitchens,"(Grocery & Gourmet Food, Pantry Staples, Cooki...","{""Package Dimensions"": ""9 x 6 x 3 inches; 7.58...",B0979H4HHW,None,None,None
369,Grocery,Snaxly - Keto Sugar Free Large Dark Chocolate ...,4.3,27,[SUGAR FREE CHOCOLATE CHIPS WITH LOW NET CARBS...,[keto chocolate chips for baking],None,Snaxly,"(Grocery & Gourmet Food, Pantry Staples, Cooki...","{""Brand"": ""Snaxly"", ""Specialty"": ""Sugar Free"",...",B0847S3BHW,None,None,None


In [17]:
df_snacks = df_meta[df_meta['title'].str.contains('snack', case=False, na=False)]
parent_asin_list_snacks = df_snacks['parent_asin'].drop_duplicates().to_list()
len(parent_asin_list_snacks)

25453

In [18]:
df_snacks.head(5)

,main_category,title,average_rating,rating_number,features,description,price,store,categories,details,parent_asin,bought_together,subtitle,author
15,Grocery,"Bulletproof Bars Collagen Protein, Chocolate C...",3.6,53,[],[],None,Bulletproof,(),"{""Package Dimensions"": ""5.8 x 4.6 x 1.8 inches...",B07Z9N3NRC,None,None,None
21,Grocery,"Peeled Snacks Organic Peas Please, Southwest S...",3.4,158,"[Baked all-natural crunchy pea snack, 3.5 vegg...","[Product Description, We believe you should fe...",None,Peeled Snacks,"(Grocery & Gourmet Food, Snacks & Sweets, Snac...","{""Product Dimensions"": ""6.5 x 2.3 x 11 inches;...",B00ZIG3DP0,None,None,None
34,Grocery,Quevos Egg White Chips - Original Low Carb Egg...,3.9,11,[THE FIRST EGG WHITE CRISPS - We knew that pot...,[],None,Quevos,"(Grocery & Gourmet Food, Snacks & Sweets, Snac...","{""Package Dimensions"": ""8.39 x 7.99 x 5.2 inch...",B09KSCP4MS,None,None,None
68,Grocery,Friendly Grains - Crunchy Rollers - Organic Ri...,4.5,400,[Made with 7 simple ingredients you can pronou...,[],23.99,Friendly Grains,"(Grocery & Gourmet Food, Snacks & Sweets, Snac...","{""Package Dimensions"": ""11.89 x 6.73 x 4.61 in...",B09S8Q6FG5,None,None,None
72,Grocery,Materra Cacao Nibs - 100% Natural Roasted Unsw...,3.6,52,[NATURALLY ROASTED - Made from naturally roast...,[],None,MATERRA,"(Grocery & Gourmet Food, Pantry Staples, Cooki...","{""Flavor"": ""Chocolate"", ""Brand"": ""MATERRA"", ""W...",B096XY398G,None,None,None


In [19]:
parent_asin_list = parent_asin_list_carb + parent_asin_list_snacks
df_healthy = df_reviews[df_reviews['parent_asin'].isin(parent_asin_list)]
df_healthy.head(5)

,rating,title,text,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
31,2.0,Weird aftertaste,"I guess I am not a fan of any ""alternative"" sw...",B07NQFHF93,B09NX8TT5H,AGKHLEW2SOWHNMFQIJGBECAF7INQ,1674767150856,0,True
103,5.0,Huge bag of crunchy goodness,These are so tasty! Nice crunch but not tooth ...,B07TZXL7FP,B0BVWMVCVP,AFJTRBXMURLHS5EGNXLUHDHIZRFQ,1635051575778,1,True
122,1.0,I never should have bought these online!,So disappointing! I love Raisinets and these ...,B089FNVZK4,B089FNVZK4,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,1659986679486,0,True
139,5.0,SO GOOD!,OMG came across this at 7-11 and thought I'd t...,B00JXY2UM0,B0779GXRXX,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,1508858480429,1,False
152,2.0,Nothing Special,After reading reviews I purchased this snack. ...,B07MVLG93G,B07MVP9VRG,AGXFEGMNVCSTSYYA5UWXDV7AFSXA,1676752273615,0,True


In [20]:
df_meta = df_meta.rename(columns={'parent_asin': 'parent_asin_meta'})
joined_df = pd.merge(df_healthy, df_meta, left_on="parent_asin", right_on="parent_asin_meta", how="left")
joined_df.head(10)

,rating,title_x,text,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,main_category,...,features,description,price,store,categories,details,parent_asin_meta,bought_together,subtitle,author
0,2.0,Weird aftertaste,"I guess I am not a fan of any ""alternative"" sw...",B07NQFHF93,B09NX8TT5H,AGKHLEW2SOWHNMFQIJGBECAF7INQ,1674767150856,0,True,Grocery,...,[ALL NEW VARIETY PACK: Contains two packs of e...,"[We love candy too, But we don’t love all the ...",23.99,SmartSweets,"(Grocery & Gourmet Food, Snacks & Sweets, Cand...","{""Package Dimensions"": ""8.54 x 5.79 x 2.2 inch...",B09NX8TT5H,None,None,None
1,5.0,Huge bag of crunchy goodness,These are so tasty! Nice crunch but not tooth ...,B07TZXL7FP,B0BVWMVCVP,AFJTRBXMURLHS5EGNXLUHDHIZRFQ,1635051575778,1,True,Grocery,...,"[A delicious, crunchy snack! Can also be added...",[Yupik’s barbecue toasted corn Nuts are savory...,12.99,Yupik,"(Grocery & Gourmet Food, Pantry Staples, Cooki...","{""Flavor"": ""Toasted Corn Nuts"", ""Brand"": ""Yupi...",B0BVWMVCVP,None,None,None
2,1.0,I never should have bought these online!,So disappointing! I love Raisinets and these ...,B089FNVZK4,B089FNVZK4,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,1659986679486,0,True,Grocery,...,[CALIFORNIA RAISINS. Grapes dried in Californi...,"[It’s snack time, grab some, Milk Chocolate Ra...",None,KMWA,"(Grocery & Gourmet Food, Snacks & Sweets, Choc...","{""Brand"": ""KMWA"", ""Item Form"": ""Beans"", ""Choco...",B089FNVZK4,None,None,None
3,5.0,SO GOOD!,OMG came across this at 7-11 and thought I'd t...,B00JXY2UM0,B0779GXRXX,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,1508858480429,1,False,Grocery,...,[🥐WHAT'S INCLUDED: 24 individually wrapped cro...,[],27.99,7 Days,"(Grocery & Gourmet Food, Breads & Bakery, Past...","{""Flavor"": ""Variety Pack (2 Flavors)"", ""Brand""...",B0779GXRXX,None,None,None
4,2.0,Nothing Special,After reading reviews I purchased this snack. ...,B07MVLG93G,B07MVP9VRG,AGXFEGMNVCSTSYYA5UWXDV7AFSXA,1676752273615,0,True,Grocery,...,[LOCALLY MADE HERE IN OREGON - Enjoy a taste o...,[],12.99,OREGON FARM FRESH SNACKS,"(Grocery & Gourmet Food, Snacks & Sweets, Snac...","{""Brand"": ""Oregon Farm Fresh Snacks"", ""Flavor""...",B07MVP9VRG,None,None,None
5,5.0,Happy,"These are great, flavor is great. Definitely w...",B01B0483TE,B098KZLL4P,AHZIJGKEWRTAEOZ673G5B3SNXEGQ,1602937979285,0,True,Grocery,...,"[BITE-SIZED CHEESY PRETZELS: Real, tangy chedd...",[Snyder's of Hanover pretzel sandwiches have r...,None,Snyder's of Hanover,"(Grocery & Gourmet Food, Breads & Bakery, Brea...","{""Package Dimensions"": ""11.73 x 11.02 x 5.91 i...",B098KZLL4P,None,None,None
6,5.0,So Yummy,Heard about these on a Podcast. They taste ama...,B07BKNVQNH,B07DYPQXY4,AH4MB6HER73XDOV6I4ICPNT24YQQ,1586879365047,0,True,Grocery,...,[DELICIOUS AND NUTRITIOUS: Our gluten-free bar...,[],25.99,This Bar Saves Lives,"(Grocery & Gourmet Food, Snacks & Sweets, Snac...","{""Package Dimensions"": ""6.18 x 5.78 x 2.63 inc...",B07DYPQXY4,None,None,None
7,5.0,Office Yummys,Great office snack gift!,B01EI0Y3IM,B09Y6BJVYF,AH4MB6HER73XDOV6I4ICPNT24YQQ,1473653272000,1,True,Grocery,...,[Snack Chest Bulk Sampler includes an assortme...,[],69.99,Snack Chest,"(Grocery & Gourmet Food, Snacks & Sweets, Snac...","{""Brand"": ""Snack Chest"", ""Package Information""...",B09Y6BJVYF,None,None,None
8,5.0,Good value,Convenient and delicious!,B01ND0XEQ4,B0777X9MFN,AHFY2QSS6PK5MHSYZFI6TXUYNPLQ,1556014695056,0,True,Grocery,...,"[Thirty 0.75-ounce snack packs, 90 calories pe...",[Our pleasantly puffed Pea Crisps are baked to...,None,Wickedly Prime,(),"{""Product Dimensions"": ""9.75 x 7.75 x 11.75 in...",B0777X9MFN,None,None,None
9,1.0,Disappointed,Something was wrong with both candies. The Kit...,B07CJW73DB,B07CJW73DB,AGMJ3EMDVL6OWBJF7CA5RGJLXN5A,1575508921799,1,True,Grocery,...,"[Contains one (1) 46.38-ounce, 85-piece bulk v...",[Enjoy a delicious chocolaty treat anywhere an...,None,HERSHEY'S,"(Grocery & Gourmet Food, Snacks & Sweets, Choc...","{""Is Discontinued By Manufacturer"": "

In [21]:
import nltk
import spacy
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess
import string

In [22]:
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fredd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
nlp = spacy.load('en_core_web_sm')

In [24]:
def preprocess_text(text):
    # Tokenize and remove punctuation
    tokens = simple_preprocess(text, deacc=True)

    # Remove stopwords and punctuation
    tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
    
    return tokens

In [25]:
joined_df['cleaned_text'] = joined_df['text'].apply(preprocess_text)
joined_df.head(10)

,rating,title_x,text,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,main_category,...,description,price,store,categories,details,parent_asin_meta,bought_together,subtitle,author,cleaned_text
0,2.0,Weird aftertaste,"I guess I am not a fan of any ""alternative"" sw...",B07NQFHF93,B09NX8TT5H,AGKHLEW2SOWHNMFQIJGBECAF7INQ,1674767150856,0,True,Grocery,...,"[We love candy too, But we don’t love all the ...",23.99,SmartSweets,"(Grocery & Gourmet Food, Snacks & Sweets, Cand...","{""Package Dimensions"": ""8.54 x 5.79 x 2.2 inch...",B09NX8TT5H,None,None,None,"[guess, fan, alternative, sweetener, keep, min..."
1,5.0,Huge bag of crunchy goodness,These are so tasty! Nice crunch but not tooth ...,B07TZXL7FP,B0BVWMVCVP,AFJTRBXMURLHS5EGNXLUHDHIZRFQ,1635051575778,1,True,Grocery,...,[Yupik’s barbecue toasted corn Nuts are savory...,12.99,Yupik,"(Grocery & Gourmet Food, Pantry Staples, Cooki...","{""Flavor"": ""Toasted Corn Nuts"", ""Brand"": ""Yupi...",B0BVWMVCVP,None,None,None,"[tasty, nice, crunch, tooth, shattering, like,..."
2,1.0,I never should have bought these online!,So disappointing! I love Raisinets and these ...,B089FNVZK4,B089FNVZK4,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,1659986679486,0,True,Grocery,...,"[It’s snack time, grab some, Milk Chocolate Ra...",None,KMWA,"(Grocery & Gourmet Food, Snacks & Sweets, Choc...","{""Brand"": ""KMWA"", ""Item Form"": ""Beans"", ""Choco...",B089FNVZK4,None,None,None,"[disappointing, love, raisinets, inedible, cho..."
3,5.0,SO GOOD!,OMG came across this at 7-11 and thought I'd t...,B00JXY2UM0,B0779GXRXX,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,1508858480429,1,False,Grocery,...,[],27.99,7 Days,"(Grocery & Gourmet Food, Breads & Bakery, Past...","{""Flavor"": ""Variety Pack (2 Flavors)"", ""Brand""...",B0779GXRXX,None,None,None,"[omg, came, across, thought, try, hooked, croi..."
4,2.0,Nothing Special,After reading reviews I purchased this snack. ...,B07MVLG93G,B07MVP9VRG,AGXFEGMNVCSTSYYA5UWXDV7AFSXA,1676752273615,0,True,Grocery,...,[],12.99,OREGON FARM FRESH SNACKS,"(Grocery & Gourmet Food, Snacks & Sweets, Snac...","{""Brand"": ""Oregon Farm Fresh Snacks"", ""Flavor""...",B07MVP9VRG,None,None,None,"[reading, reviews, purchased, snack, little, v..."
5,5.0,Happy,"These are great, flavor is great. Definitely w...",B01B0483TE,B098KZLL4P,AHZIJGKEWRTAEOZ673G5B3SNXEGQ,1602937979285,0,True,Grocery,...,[Snyder's of Hanover pretzel sandwiches have r...,None,Snyder's of Hanover,"(Grocery & Gourmet Food, Breads & Bakery, Brea...","{""Package Dimensions"": ""11.73 x 11.02 x 5.91 i...",B098KZLL4P,None,None,None,"[great, flavor, great, definitely, buy]"
6,5.0,So Yummy,Heard about these on a Podcast. They taste ama...,B07BKNVQNH,B07DYPQXY4,AH4MB6HER73XDOV6I4ICPNT24YQQ,1586879365047,0,True,Grocery,...,[],25.99,This Bar Saves Lives,"(Grocery & Gourmet Food, Snacks & Sweets, Snac...","{""Package Dimensions"": ""6.18 x 5.78 x 2.63 inc...",B07DYPQXY4,None,None,None,"[heard, podcast, taste, amazing, love, love, c..."
7,5.0,Office Yummys,Great office snack gift!,B01EI0Y3IM,B09Y6BJVYF,AH4MB6HER73XDOV6I4ICPNT24YQQ,1473653272000,1,True,Grocery,...,[],69.99,Snack Chest,"(Grocery & Gourmet Food, Snacks & Sweets, Snac...","{""Brand"": ""Snack Chest"", ""Package Information""...",B09Y6BJVYF,None,None,None,"[great, office, snack, gift]"
8,5.0,Good value,Convenient and delicious!,B01ND0XEQ4,B0777X9MFN,AHFY2QSS6PK5MHSYZFI6TXUYNPLQ,1556014695056,0,True,Grocery,...,[Our pleasantly puffed Pea Crisps are baked to...,None,Wickedly Prime,(),"{""Product Dimensions"": ""9.75 x 7.75 x 11.75 in...",B0777X9MFN,None,None,None,"[convenient, delicious]"
9,1.0,Disappointed,Something was wrong with both candies. The Kit...,B07CJW73DB,B07CJW73DB,AGMJ3EMDVL6OWBJF7CA5RGJLXN5A,1575508921799,1,True,Grocery,...,[Enjoy a delicious chocolaty treat anywhere an...,None,HERSHEY'S,"(Grocery & Gourmet Food, Snacks & Sweets, Choc...","{""Is Discontinued By Manufacturer"": ""No"", ""Pro...",B07CJW73DB,None,None,None,"[something, wrong, candies, kit, kat, broken, ...

In [26]:
processed_reviews = joined_df['cleaned_text'].tolist()

In [27]:
from tqdm import tqdm
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore
import logging

In [28]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [29]:
if isinstance(joined_df['cleaned_text'].iloc[0], list):
    processed_reviews = joined_df['cleaned_text'].tolist()
else:
    raise ValueError("Ensure 'processed_reviews' contains lists of tokens")

In [30]:
dictionary = corpora.Dictionary(processed_reviews)

2024-12-08 14:03:38,182 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2024-12-08 14:03:38,426 : INFO : adding document #10000 to Dictionary<11745 unique tokens: ['alternative', 'away', 'brilliant', 'dislike', 'fan']...>
2024-12-08 14:03:38,612 : INFO : adding document #20000 to Dictionary<15034 unique tokens: ['alternative', 'away', 'brilliant', 'dislike', 'fan']...>
2024-12-08 14:03:38,791 : INFO : adding document #30000 to Dictionary<17672 unique tokens: ['alternative', 'away', 'brilliant', 'dislike', 'fan']...>
2024-12-08 14:03:38,963 : INFO : adding document #40000 to Dictionary<19889 unique tokens: ['alternative', 'away', 'brilliant', 'dislike', 'fan']...>
2024-12-08 14:03:39,138 : INFO : adding document #50000 to Dictionary<21511 unique tokens: ['alternative', 'away', 'brilliant', 'dislike', 'fan']...>
2024-12-08 14:03:39,333 : INFO : adding document #60000 to Dictionary<23280 unique tokens: ['alternative', 'away', 'brilliant', 'dislike', 'fan']...>
2024-12-08 14

In [31]:
corpus = [dictionary.doc2bow(review) for review in processed_reviews]
total_docs = len(corpus)

In [32]:
def custom_lda_training(corpus, dictionary, num_topics, passes, chunksize):
    num_iterations = passes

    with tqdm(total=total_docs * num_iterations, desc="LDA Training Progress") as pbar:
        # Initialize the LDA Multicore model
        lda_model = LdaMulticore(
            corpus=corpus,
            id2word=dictionary,
            num_topics=num_topics,
            random_state=100,
            chunksize=chunksize,
            passes=1,
            workers=1,
            alpha='symmetric',
            per_word_topics=True,
            batch=True
        )

        # Manually iterate over the passes and update the progress bar
        for pass_ in range(num_iterations):
            for i in range(0, total_docs, chunksize):
                chunk = corpus[i:i + chunksize]
                lda_model.update(chunk)
                pbar.update(len(chunk))

    return lda_model

In [33]:
lda_model = custom_lda_training(corpus, dictionary, num_topics=20, passes=5, chunksize=3000)

LDA Training Progress:   0%|          | 0/6383545 [00:00<?, ?it/s]2024-12-08 14:09:05,721 : INFO : using symmetric alpha at 0.05
2024-12-08 14:09:05,722 : INFO : using symmetric eta at 0.05
2024-12-08 14:09:05,738 : INFO : using serial LDA version on this node
2024-12-08 14:09:05,883 : INFO : running batch LDA training, 20 topics, 1 passes over the supplied corpus of 1276709 documents, updating every 1276709 documents, evaluating every ~1276709 documents, iterating 50x with a convergence threshold of 0.001000
2024-12-08 14:09:05,884 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2024-12-08 14:09:05,887 : INFO : training LDA model using 1 processes
2024-12-08 14:09:08,335 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #3000/1276709, outstanding queue size 1
2024-12-08 14:09:08,354 : INFO : PROGRESS: pass 0, dispatched chunk #1 = documents up to #6000/1276709, outstanding queue size 2


MemoryError: 